# 要約 
このJupyterノートブックは、Kaggleの「20 Questions」コンペティションに向けて、Hugging Faceの8Bまたはそれ以下のLLMを活用したエージェントを開発するためのスターターコードです。ノートブックは、質問者と回答者の役割を持つエージェントを設計することを目指しており、質問者は特定のキーワードを効率的に推測するためにあらかじめ設定された質問を用いることができます。一方、回答者はHugging FaceのLlama 8B LLMを使用して、未知の質問に対する「はい」または「いいえ」の回答を生成する必要があります。

主な処理として以下のステップが含まれます：

1. **ライブラリのインストール**: 必要なPythonライブラリ（`accelerate`、`bitsandbytes`など）が指定したパスにインストールされます。
   
2. **モデルのダウンロードと保存**: Llama-3-Smaug-8Bモデルが4bit形式でダウンロードされ、保存されます。このモデルは、追加データでファインチューニングされた8BのLlama3モデルです。

3. **エクスプロラトリーデータ分析（EDA）**: キーワードに対するLLMの応答を確認するためのセクションが設けられています。ここでは、回答者にカスタマイズされたプロンプトが生成されます。

4. **エージェントの作成**: メインのエージェントファイル`main.py`に、エージェントのロジックを実装します。質問や推測の処理が定義されており、各ラウンドごとに情報をもとに質問を更新します。

5. **Kaggle環境での実行**: KaggleのAPIを使用して、エージェントを実行し、シミュレーションを通じてエージェントのパフォーマンスを評価します。

このノートブックを通じて、作者は質問者と回答者の双方の役割を担うエージェントの設計と実装を進め、Kaggleの「20 Questions」ゲームにおいて効率的な推測を行う方法を探求しています。使用しているライブラリは主に`transformers`であり、これによりLLMのモデルの読み込みと生成のプロセスがスムーズに行われます。

---


# 用語概説 
以下は、Jupyter Notebook内の専門用語に対する簡単な解説です。初心者がつまずきやすいと思われるマイナーなトピックや、このノートブック特有のドメイン知識に焦点を当てています。

1. **Hugging Face**: 自然言語処理（NLP）のためのオープンソースライブラリを提供する企業。特にTransformerモデルの事前学習済みモデルの提供で知られている。

2. **LLM (Large Language Model)**: 大規模な言語モデルの略。膨大なデータでトレーニングされ、自然言語の理解と生成ができるモデル。

3. **二分探索 (Binary Search)**: ソートされたデータセットにおいて、対象の値を効果的に見つけるためのアルゴリズム。データを半分に分割していくつかのステップで最終的な検索結果に到達する。

4. **4bit量子化 (4bit Quantization)**: モデルのパラメータを4ビットの精度で表現する技術。モデルのサイズを大幅に削減し、メモリ使用量を減らす効果がある。

5. **ヒューリスティック (Heuristic)**: 問題解決のために経験則に基づいて行う手法。必ずしも最適解を提供するわけではないが、実用的で効率的な解決策を見つけられることが多い。

6. **トークナイザー (Tokenizer)**: テキストをモデルに入力するために、単語や文などの意味のある単位（トークン）に分割するツール。モデルが扱いやすいフォーマットに変換する。

7. **CUDA**: NVIDIAの並列コンピューティングアーキテクチャ。GPUを用いた計算を効率的に行うためのプログラミングモデル。

8. **pad_token_id**: モデルの入力データを揃えるために使用されるパディングトークンのID。入力シーケンスの長さを揃えるために使われる。

9. **generate**: モデルにテキストを生成させるメソッド。特定の入力に基づいて次のトークンを予測する。

10. **KaggleのAPI**: Kaggleプラットフォームでのデータの取得やノートブックの実行などをプログラムから行うためのインターフェース。

11. **エージェント (Agent)**: コンペティションのルールのもとで振る舞うプログラム。特定の環境内で自動的に行動し、相手と対戦する。

12. **obs (Observation)**: エージェントが周囲の環境から取得する情報。例えば、現在のラウンドの状態や相手の質問、過去の回答などが含まれる。

これらの用語は、初心者がノートブックの内容を理解する上で役立つでしょう。特に、モデルの設定や出力に関わる部分については、実務経験がないと理解が難しい場合があるため、詳しく説明しました。

---


# Hugging Face Llama 8B LLM のスターターコード
これは、Kaggleの「20 Questions」コンペティションに向けたHugging Faceの8Bまたはそれ以下のLLM用のスターターコードです。我々のエージェントは、`質問者`と`回答者`の両方として機能する必要があります。質問者は、作成した新機能を基にリストや潜在的なキーワードから二分探索を行います（更新前）。**このノートブックは、キーワードが場所だけだった時に作成されました。**

このノートブックを実行する前に、我々はすべての公的LB（更新前）の単語を含むCSVファイルを作成し、**場所**キーワードを検索するために使用できるいくつかの追加列を用意しました。追加した列は`大陸`と`最初の文字`です。回答者としてHugging FaceのLlama 8B LLMを使用します。

質問者の「尋ねる」役割は**LLMを使用する必要はありません**が、検索を絞り込むためのあらかじめ決められた質問のセットであれば大丈夫です。また、質問者の「推測する」役割も、ヒューリスティックな論理を使用することができ、LLMである必要はありません。回答者は**LLMである必要があります**。なぜなら、他のKaggleチームからの未知の質問に答える必要があるからです。

# 必要なライブラリのインストール
必要なパッケージを`/tmp/submission/lib`パスにインストールします。このパスは、提出ファイルにtarballとしてまとめられます。


In [ ]:
!mkdir /tmp/submission

In [ ]:
%%time
import os,sys
os.system("pip install -U -t /tmp/submission/lib accelerate")
os.system("pip install -i https://pypi.org/simple/ -U -t /tmp/submission/lib bitsandbytes")
os.system("pip cache purge")
sys.path.insert(0, "/tmp/submission/lib")

# LLMモデルのダウンロードと保存
モデルを`/tmp/submission/weights`にダウンロードして保存します。ディスクとメモリ使用量を最小限に抑えるために、モデルを4bitで保存します。このモデルも提出ファイルにtarballとしてまとめられます。

このノートブックでは、「Llama-3-Smaug-8B」を使用します。これは、追加のデータでファインチューニングされたLlama-3-8Bです。詳細はHuggingFaceで確認できます[こちら][1]

[1]: https://huggingface.co/abacusai/Llama-3-Smaug-8B


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

In [ ]:
model_name = "abacusai/Llama-3-Smaug-8B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quanty_type = "fp4", 
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quanty = True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    torch_dtype = torch.float16,
    device_map = "auto",
    trust_remote_code = True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
model.save_pretrained("/tmp/submission/weights")
tokenizer.save_pretrained("/tmp/submission/weights")

# LLMモデルのEDA
キーワードに関する質問に対するLLMの回答を確認してみましょう。このセクションを使用して、`回答者`に表示するカスタマイズされたプロンプトを決定します。


In [ ]:
import pandas as pd
keywords = pd.read_csv("/kaggle/input/updated-kaggle-keywords/keywords_v2.csv")
keywords.head(1)

In [ ]:
pad_token_id = tokenizer.pad_token_id
if pad_token_id is None:
    pad_token_id = tokenizer.eos_token_id

In [ ]:
question = f"秘密の単語は国ですか？"

keyword = "venezuela"
if keyword in keywords.keyword.values:
    row = keywords.loc[keywords.keyword==keyword].iloc[0]
    category = row.category #"名所"
    continent = row.continent #"北アメリカ"
    negate = {
        "city":"国ではありません。名所でもありません。",
        "country":"都市ではありません。名所でもありません。",
        "landmark":"都市ではありません。国でもありません。",
    }
    prompt = f"我々は20の質問をプレイしています。キーワードは{keyword}です。これは{category}です。{negate[category]} この単語の最初の文字は{keyword[0]}です。この{category}は{continent}にあります。{question}"
else:
    prompt = f"我々は20の質問をプレイしています。キーワードは{keyword}です。これは物体です。都市ではありません。国でもありません。名所でもありません。この単語の最初の文字は{keyword[0]}です。{question}"

messages = [
    {"role": "system", "content": "次の質問には「はい」または「いいえ」のみで答えてください。それ以外は答えないでください。"},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to('cuda')

generated_ids = model.generate(
    model_inputs.input_ids,
    attention_mask = model_inputs.attention_mask,
    pad_token_id=pad_token_id,
    max_new_tokens=1
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
if not "yes" in response.lower(): response = "no"
else: response = "yes"

print(f"キーワードが '{keyword}' の時")
print(f"質問者が尋ねる: '{question}'")
print(f"プロンプトは: {prompt}")
print(f"私たちのモデルの回答: '{response}'")

In [ ]:
import gc, torch
del model, tokenizer, model_inputs, generated_ids, response
gc.collect()
torch.cuda.empty_cache()

# エージェントの作成
エージェントファイルを`main.py`として保存します。我々のエージェントは、`def agent_fn(obs, cfg)`という関数を実装する必要があります。


In [ ]:
!cp /kaggle/input/updated-kaggle-keywords/keywords_v2.csv /kaggle/working
!cp /kaggle/input/updated-kaggle-keywords/keywords_v2.csv /tmp/submission

In [ ]:
%%writefile /tmp/submission/main.py

################
# 提出かコミットかを決定
import os
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
VERBOSE = False
if not os.path.exists(KAGGLE_AGENT_PATH + "weights"):
    KAGGLE_AGENT_PATH = "/tmp/submission/"
    VERBOSE = True

#################
# メモリにモデルをロード
import sys, torch
from transformers import AutoTokenizer, AutoModelForCausalLM
sys.path.insert(0, f"{KAGGLE_AGENT_PATH}lib")
model = AutoModelForCausalLM.from_pretrained(
    f"{KAGGLE_AGENT_PATH}weights/",
    torch_dtype = torch.float16,
    device_map = "auto",
    trust_remote_code = True,
)
tokenizer = AutoTokenizer.from_pretrained(f"{KAGGLE_AGENT_PATH}weights/")

##############
# 質問者としての二分探索
import pandas as pd, numpy as np
keywords = pd.read_csv(KAGGLE_AGENT_PATH + "keywords_v2.csv")
keywords['guess'] = 0

categories = ["city","country","landmark"]
#np.random.shuffle(categories)
category_yes = []
category_no = []
cat_guess = 0

continents = ["Europe","Asia","North America","Africa","South America","Australia"]
#np.random.shuffle(continents)
continent_yes = []
continent_no = []
con_guess = 0

first_letters = []
first_letter_yes = []
first_letter_no = []
let_guess = 0
extra_guess = ""

###############
# LLMモデルとしての回答者
def get_yes_no(question,keyword):
    global keywords, VERBOSE
    
    if keyword in keywords.keyword.values:
        row = keywords.loc[keywords.keyword==keyword].iloc[0]
        category = row.category #"名所"
        continent = row.continent #"北アメリカ"
        negate = {
            "city":"国ではありません。名所でもありません。",
            "country":"都市ではありません。名所でもありません。",
            "landmark":"都市ではありません。国でもありません。",
        }
        prompt = f"我々は20の質問をプレイしています。キーワードは{keyword}です。これは{category}です。{negate[category]} この単語の最初の文字は{keyword[0]}です。この{category}は{continent}にあります。{question}"
    else:
        prompt = f"我々は20の質問をプレイしています。キーワードは{keyword}です。これは物体です。都市ではありません。国でもありません。名所でもありません。この単語の最初の文字は{keyword[0]}です。{question}"
    
    messages = [
        {"role": "system", "content": "次の質問には「はい」または「いいえ」のみで答えてください。それ以外は答えないでください。"},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to('cuda')
    
    pad_token_id = tokenizer.pad_token_id
    if pad_token_id is None:
        pad_token_id = tokenizer.eos_token_id
        
    generated_ids = model.generate(
        model_inputs.input_ids,
        attention_mask = model_inputs.attention_mask,
        pad_token_id = pad_token_id,
        max_new_tokens=1
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    if not "yes" in response.lower(): response = "no"
    else: response = "yes"
        
    if VERBOSE:
        print(f"### {prompt}")
        
    return response

############
# メインエージェント関数
def agent_fn(obs, cfg):
    global keywords, extra_guess, VERBOSE
    global categories, category_yes, category_no, cat_guess
    global continents, continent_yes, continent_no, con_guess
    global first_letters, first_letter_yes, first_letter_no, let_guess
        
    # レスポンスを生成
    if obs.turnType == "ask":
        
        if (cat_guess<3)&(len(category_yes)==0):
            response = f"キーワードは{categories[cat_guess]}の名前ですか？"
            cat_guess += 1
        elif (con_guess<6)&(len(continent_yes)==0):
            category = "場所"
            if len( category_yes )==1: 
                category = category_yes[0]
            response = f"その{category}は{continents[con_guess]}にありますか？"
            con_guess += 1
        else:
            IDX = keywords.category.isin( category_yes )
            IDX = IDX & (keywords.continent.isin( continent_yes ))
            first_letters = list(keywords.loc[IDX,"first_letter"].value_counts().index.values)
            if let_guess < len(first_letters):
                response = f"キーワードの最初の文字は{first_letters[let_guess]}ですか？"
            else:
                IDX = keywords.guess == 0
                if len(category_yes)>0: IDX = IDX & (keywords.category.isin(category_yes))
                if len(category_no)>0: IDX = IDX & (~keywords.category.isin(category_no))
                if len(continent_yes)>0: IDX = IDX & (keywords.continent.isin(continent_yes))
                if len(continent_no)>0: IDX = IDX & (~keywords.continent.isin(continent_no))
                if len(first_letter_yes)>0: IDX = IDX & (keywords.first_letter.isin(first_letter_yes))
                if len(first_letter_no)>0: IDX = IDX & (~keywords.first_letter.isin(first_letter_no))
                try:
                    guess = keywords.loc[IDX].sample(1).index.values[0]
                    keywords.loc[guess,'guess'] = 1
                    response = keywords.loc[guess,"keyword"]
                except:
                    response = np.random.choice( keywords.keyword.values )
                extra_guess = response
                response = f"それは{response}ですか？"
            let_guess += 1
            
    elif obs.turnType == "guess":
        
        category_yes = []
        category_no = []
        for k in range(cat_guess):
            if obs.answers[k]=="yes":
                category_yes.append( categories[k] )
            else:
                category_no.append( categories[k] )
        if (cat_guess==3)&(len(category_yes)==0):
            category_yes = ["city","country","landmark"]
            category_no = []
            
        continent_yes = []
        continent_no = []
        for k in range(con_guess):
            if obs.answers[k+cat_guess]=="yes":
                continent_yes.append( continents[k] )
            else:
                continent_no.append( continents[k] )
        if (con_guess==6)&(len(continent_yes)==0):
            continent_yes = ["Europe","Asia","North America","Africa","South America","Australia"]
            continent_no = []
            
        first_letter_yes = []
        first_letter_no = []
        for k in range(let_guess):
            if k >= len(first_letters): continue
            if obs.answers[k+cat_guess+con_guess]=="yes":
                first_letter_yes.append( first_letters[k] )    
            else:
                first_letter_no.append( first_letters[k] ) 
                
        IDX = keywords.guess == 0
        if len(category_yes)>0: IDX = IDX & (keywords.category.isin(category_yes))
        if len(category_no)>0: IDX = IDX & (~keywords.category.isin(category_no))
        if len(continent_yes)>0: IDX = IDX & (keywords.continent.isin(continent_yes))
        if len(continent_no)>0: IDX = IDX & (~keywords.continent.isin(continent_no))
        if len(first_letter_yes)>0: IDX = IDX & (keywords.first_letter.isin(first_letter_yes))
        if len(first_letter_no)>0: IDX = IDX & (~keywords.first_letter.isin(first_letter_no))
            
        try:
            guess = keywords.loc[IDX].sample(1).index.values[0]
            keywords.loc[guess,'guess'] = 1
            response = keywords.loc[guess,"keyword"]
        except:
            response = np.random.choice( keywords.keyword.values )
            
        if (let_guess>0)&(let_guess>=len(first_letters))&(obs.answers[-1]=="yes"):
            response = extra_guess
        
    else: #obs.turnType == "answer"
        if obs.keyword.lower() in obs.questions[-1].lower():
            response = "yes"
        else:
            response = get_yes_no(obs.questions[-1], obs.keyword)
            
    # 役割を表示
    if VERBOSE: 
        if obs.turnType == "answer": 
            print(f"チーム2 - 回答者 - ### エージェント LLAMA 8B ###")
        else:
            print(f"\nチーム2 - 質問者 - ### エージェント LLAMA 8B ###")
        print(f"OUTPUT = '{response}'")

    return response

# 提出用のターボールを保存
ライブラリ、モデル、main.pyをターボールに保存します。


In [ ]:
!apt install pigz pv > /dev/null

In [ ]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf /kaggle/working/submission.tar.gz -C /tmp/submission .

# KaggleのAPIのEDA
以下にKaggleのAPIのEDAを示します。我々はランダムに選ばれた3つのエージェントと共にLlama 8B LLMエージェントを実行します。

Kaggleの20 Questionsコンペティションでは、各挑戦は4つのKaggleチームが参加します。2つのチームが対戦形式です。このグループチームでは、「質問者」（4チームのうち1チーム）と「回答者」（4チームのうち1チーム）がいます。質問者の役割は2つ（すなわち、尋ねると推測する）。回答者の役割は1つ（すなわち、答える）です。

各ラウンドで、質問者が質問をし、その後グループの仲間である回答者が「はい」か「いいえ」で答えます。最後に質問者が単語を推測します。合計20ラウンド行われます。


In [ ]:
import pandas as pd

# DataFrameの表示
df = pd.read_csv("/kaggle/input/updated-kaggle-keywords/keywords_v2.csv")
n = df.category.nunique()
print(f"\n古い公的LB（キーワード更新前）には、{len(df)}個の可能なキーワードと{n}のカテゴリがあります。")
print("以下は、ランダムに選ばれた20の例です:\n")
df.sample(20, random_state=42)

In [ ]:
%%writefile /kaggle/working/agent1.py

import pandas as pd, numpy as np
keywords = pd.read_csv("keywords_v2.csv").keyword.values

def agent_fn(obs, cfg):
    global keywords
    
    # ラウンド番号を表示
    k = len( obs.questions )
    if obs.turnType == "ask":
        print()
        print("#"*25)
        print(f"### ラウンド {k+1}")
        print("#"*25)

    # エージェント名とJSON入力を表示
    name = "チーム1 - 質問者 - エージェント ランダム"
    print(f"\n{name}\nINPUT =",obs)
    
    # レスポンスを生成
    keyword = np.random.choice(keywords)
    if obs.turnType == "ask":
        response = f"それは{keyword}ですか？"
    else: #obs.turnType == "guess"
        response = keyword
        if obs.answers[-1] == "yes":
            response = obs.questions[-1].rsplit(" ",1)[1][:-1]
    print(f"OUTPUT = '{response}'")

    return response

In [ ]:
%%writefile /kaggle/working/agent2.py

import numpy as np

def agent_fn(obs, cfg):
    
    # エージェント名とJSON入力を表示
    name = "チーム1 - 回答者 - エージェント ランダム"
    print(f"\n{name}\nINPUT =",obs)
    
    # レスポンスを生成
    response = "no"
    #response = np.random.choice(["yes","no"])
    if obs.keyword.lower() in obs.questions[-1].lower():
        response = "yes"
    print(f"OUTPUT = '{response}'")

    return response

In [ ]:
%%writefile /kaggle/working/agent3.py

import pandas as pd, numpy as np
keywords = pd.read_csv("keywords_v2.csv").keyword.values

def agent_fn(obs, cfg):
    global keywords
    
    # エージェント名とJSON入力を表示
    name = "チーム2 - 質問者 - エージェント ランダム"
    print(f"\n{name}\nINPUT =",obs)
    
    # レスポンスを生成
    keyword = np.random.choice(keywords)
    if obs.turnType == "ask":
        response = f"それは{keyword}ですか？"
    else: #obs.turnType == "guess"
        response = keyword
        if obs.answers[-1] == "yes":
            response = obs.questions[-1].rsplit(" ",1)[1][:-1]
    print(f"OUTPUT = '{response}'")

    return response

In [ ]:
%%writefile /kaggle/working/agent4.py

import numpy as np

def agent_fn(obs, cfg):
    
    # エージェント名とJSON入力を表示
    name = "チーム2 - 回答者 - エージェント ランダム"
    print(f"\n{name}\nINPUT =",obs)
    
    # レスポンスを生成
    response = "no"
    #response = np.random.choice(["yes","no"])
    if obs.keyword.lower() in obs.questions[-1].lower():
        response = "yes"
    print(f"OUTPUT = '{response}'")

    return response

In [ ]:
!pip install -q pygame

In [ ]:
LLAMA_AS_QUESTIONER = True
LLAMA_AS_ANSWERER = True

from kaggle_environments import make
env = make("llm_20_questions", debug=True)

# チーム1
agent1 = "/kaggle/working/agent1.py"
agent2 = "/kaggle/working/agent2.py"

# チーム2 - 質問者
agent3 = "/kaggle/working/agent3.py"
if LLAMA_AS_QUESTIONER:
    agent3 = "/tmp/submission/main.py"
    
# チーム2 - 回答者
agent4 = "/kaggle/working/agent4.py"
if LLAMA_AS_ANSWERER:
    agent4 = "/tmp/submission/main.py"
    
env.reset()
log = env.run([agent1, agent2, agent3, agent4])

import gc, torch
del make, env, log
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# シミュレーションエージェントを削除する。提出しないため
!rm agent1.py agent2.py agent3.py agent4.py keywords_v2.csv

---

# コメント

> ## shiv_314
> 
> このノートブックは素晴らしい！8Bパラメータのモデルを上限として使用できるということをどのように思いついたのですか？推定を通じて説明してもらえますか？
> 
> 
> > ## Chris Deotteトピック作成者
> > 
> > 1xT4 GPUを使用すると、実際には17Bモデルを推論できます（なぜなら、私はここで34Bモデルを2xT4で推論する方法を示しました）。17Bモデルから始めると、4bit量子化の後、ディスク上で10GBになり、1xT4の16GB VRAMに簡単に収まります。（更新：おそらく20Bモデルを推論できるかもしれません）
> > 
> > 
> > 


---

> ## Istvan Habram
> 
> エージェントが`def agent_fn(obs, cfg)`という関数を実装する必要があることはどこに記載されていますか？
> 
> 
> > ## Chris Deotteトピック作成者
> > 
> > これはKaggle管理者のスターターノートブック[こちら](https://www.kaggle.com/code/ryanholbrook/llm-20-questions-starter-notebook)や、KaggleのGitHub[こちら](https://github.com/Kaggle/kaggle-environments/tree/master/kaggle_environments/envs/llm_20_questions)を読むことで学びます。これには、私たちのエージェントを実行するコードが示されています。
> > 
> > 
> > 
> > > ## Istvan Habram
> > > 
> > > 今はっきりしました。迅速な回答をありがとう！
> > > 
> > > 
> > > 


---

> ## Toshi
> 
> すばらしいノートブック！
> 
> これを実行したところ、上記のエラーが発生しました
> 
> ImportError: numpy>=1.17,<2.0がこのモジュールの正常な機能に必要ですが、numpy==2.0.0が見つかりました。
> 
> 試してください: pip install transformers -U または、git mainで作業している場合はpip install -e '.[dev]'を実行してください。
> 
> このエラーは見つかりましたか？
> 
> 
> > ## Chris Deotteトピック作成者
> > 
> > いいえ、そのエラーは見たことがありません。ノートブックをそのまま実行していますか？それとも何か変更していますか？ノートブックオプションの環境を「元の状態に固定」に設定していますか？それともKaggleの最新のドッカーを使用していますか？Kaggleの最新がエラーを引き起こしている場合は、「元の状態に固定」を使用する必要があります。
> > 
> > もし何か変更して、numpy==2.0.0が気に入らない原因となっている場合、古いnumpyをインストールすることができます。例: pip install numpy==1.17
> > 
> > 
> > 
> > > ## FullEmpty
> > > >[@cdeotte](https://www.kaggle.com/cdeotte)このノートブックに感謝 - あなたの投稿やコメントから多くを学んでいます。私はKaggleノートブックをマスターしていないが、このノートブックはコピーして実行しますが、ダウンロードして取り込んで実行すると上記のエラーが発生します。「元の状態に固定」でさえ。
> > > >しかし、以下のいずれかで問題を解決できます:
> > > >
> > > > !rm -rf /tmp/submission/lib/numpy-2.0.1.dist-info
> > > > または、好ましくは、
> > > > 
> > > > os.system("pip install accelerate==0.33.0 bitsandbytes==0.43.2 huggingface_hub==0.24.2 -t /tmp/submission/lib")
> > > >
> > > > 


---

> ## Kartikeya Pandey
> > 素晴らしいノートブック。私は多くを学びました。
> 
> 
> 


---

> ## El haddad Mohamed
> 
> 素晴らしいノートブック！とても整理されていて、有益です。👍
> 
> 


---

> ## ISAKA Tsuyoshi
> 
> こんにちは[@cdeotte](https://www.kaggle.com/cdeotte)。このような素晴らしいノートブックを共有してくれてありがとう！
> 
> このノートブックで使用されているモデル(Llama-3-Smaug-8B)についてもっと教えてもらえますか？これは、llama-3モデルよりも優れていますか？
> 
> > ## Chris Deotteトピック作成者
> > 
> > Llama-3-Smaug-8BはLlama-3-8B-Instructに追加のファインチューニングを施したモデルです。いくつかの点でLlama-3-8B-Instructよりも優れていますが、おそらくいくつかの点で劣っています。Smaugがどのように訓練されたかについての研究論文は[こちら](https://arxiv.org/abs/2402.13228)で説明されています。このモデルを発見したのは、ある時点でこのモデルが通常のLlamaよりも高いOpen LLM Leaderboardのスコアを持っていたからです。
> > 
> > もう一つ、Open LLM Leaderboardで良好な結果を出すモデルはjondurbin/bagel-7b-v0.5[こちら](https://huggingface.co/jondurbin/bagel-7b-v0.5)です。これはMistral-7Bに追加のファインチューニングを施したものです。Bagelがどうやって訓練されたのかは、[こちら](https://github.com/jondurbin/bagel)で説明されています。
> > 
> > さらに、Open LLM Leaderboardで良好な結果を出すモデルはQwen/Qwen2-7B-Instruct[こちら](https://huggingface.co/Qwen/Qwen2-7B-Instruct)です。私はQwen2モデルの大ファンです。このv2シリーズは先月リリースされたばかりで、すぐにトップのOpen LLM Leaderboardモデルになりました。
> > 
> > （完全を期すために古典的なモデルとしてLlama3、Mistral、Gemma2、Phi3を挙げます）
> > 
> > 
> > 
> > > ## ISAKA Tsuyoshi
> > >  > 詳細な説明をありがとうございます！たくさんのオプションがありますね。
> > > 
> > > 
